AUTO SCRAPER:
Auto scraper is used to fetch the reviews from the given URL. We have to provie a wishlist like what values we have to scrap from the webpage. In given code, the wishlist contain the review title or how much people find it useful. We can also retrieve the star rating to enhance our project.

In [4]:
from autoscraper import AutoScraper
import pandas as pd

def scrape(x):
  amazon_url = "https://www.amazon.com/Maybelline-SuperStay-Matte-Liquid-Lipstick/product-reviews/B074VD4X86/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

  wish_list = ['Love it!', '362 people found this helpful']

  scraper_1 = AutoScraper()
  result = scraper_1.build(amazon_url, wish_list)
  review_title_key=scraper_1.get_result_similar(amazon_url, grouped= True)
  keys = list(review_title_key.keys())
  title_key = keys[0]
  scraper_1.set_rule_aliases({title_key: 'Review_Title'})
  scraper_1.keep_rules([title_key])
  scraper_1.save('Amazon-Reviews')
  scraper_1.load('Amazon-Reviews')
  link = x
  result_1 = []
  for i in range(1, 20):
    result = scraper_1.get_result_similar(
        link+'&pageNumber='+str(i), group_by_alias=True)
    result_1.append(result['Review_Title'])
  reviews = []
  for i in range(len(result_1)):
    for j in range(len(result_1[i])):
      reviews.append(result_1[i][j])
  review_dict = {'prodct_review': reviews}
  df = pd.DataFrame(review_dict)
  df.to_csv('product_reviews.csv')
  return df


First we define wishlist for autoscraper argument, build function takes URL and wishlist for scraping data, get_result_similar scrap data in a form of dictionary, after that set_rule_aliases set the title keys. We save our model and load it to try on different web pages. For instance we scrape 20 pages and save the dataframe tp csv file.

In [5]:
scrape("https://www.amazon.com/Maybelline-SuperStay-Matte-Liquid-Lipstick/product-reviews/B074VD4X86/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews")


,prodct_review
0,Love it!
1,Gotta collect them all!
2,A.D.D.I.C.T.E.D.❣️
3,Day two
4,"Excellent formula, but be careful in color sel..."
...,...
84,Will buy again!!
85,This Stuff Stays Put
86,Love the color!
87,Maybelline Got It Right!


In [ ]:
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import pandas as pd
nltk.download('vader_lexicon')


def parse_values(x):
    if x < 0:
       return 'negative'
    elif x == 0:
       return 'neutral'
    else:
       return 'positive'


sid = SentimentIntensityAnalyzer()
df = pd.read_csv('product_reviews.csv')
df['prodct_review'].isnull().sum()
df['scores'] = df['prodct_review'].apply(
    lambda review: sid.polarity_scores(review))
df['compound'] = df['scores'].apply(lambda score_dict: score_dict['compound'])
df['comp_score'] = df['compound'].apply(parse_values)
df.to_csv('sentiments_1.csv')
